<a href="https://colab.research.google.com/github/iso-ai/isopro_examples/blob/main/examples/workflow_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automating a Meme Generator with Workflow Simulation
This notebook demonstrates how to use isopro.workflow_simulation to automate a meme generation workflow. We'll train an agent to:

1. Navigate a meme generator website
2. Upload images
3. Add captions
4. Generate and download memes

And do it all automatically!

## Setup
First, let's import our required libraries and set up our environment:

In [ ]:
!pip3 install isopro --upgrade
!pip install stable-baselines3
!pip install anthropic
!pip install python-dotenv
!pip install iso-adverse
!pip install isozero
!pip install tiktoken

In [ ]:
import os
from pathlib import Path
from isopro import (
    WorkflowSimulator,
    AgentConfig,
    VisualizationConfig,
    ValidationConfig
)
import matplotlib.pyplot as plt
from IPython.display import Image, HTML
from google.colab import userdata

## Configuration
Let's create a fun configuration for our meme generator automation:

In [ ]:
# Create output directory for our memes
output_dir = Path("meme_generator.mp4")
output_dir.mkdir(exist_ok=True)

# Configure our agent with some fun parameters
agent_config = AgentConfig(
    learning_rate=3e-4,  # Not too fast, not too slow - just right for meme making
    pretrain_epochs=10,  # Give it some time to learn the art of memes
    use_demonstration=True,  # Learn from the meme masters
    use_reasoning=True,  # Think before you meme
    reward_threshold=0.8  # High standards for our memes!
)

# Set up visualization so we can watch the magic happen
viz_config = VisualizationConfig(
    show_ui_elements=True,  # See what the agent sees
    show_cursor=True,  # Watch the cursor dance
    show_actions=True,  # Understand what's happening
    save_frames=True,  # Save the best moments
    real_time_display=True  # Watch it live!
)

# Define what makes a successful meme
validation_config = ValidationConfig.from_dict({
    "success_criteria": [
        "image_uploaded",
        "captions_added",
        "meme_generated",
        "meme_downloaded"
    ],
    "error_tolerance": 0.1  # Some memes are meant to be a little off...
})

## Recording a Demonstration
Before we can train our agent, we need to show it how to make memes. Here's how we record a demonstration:

In [ ]:
# Initialize our simulator
simulator = WorkflowSimulator(
    video_path="/content/meme-generator.mp4",  # Your recorded workflow video
    agent_config=agent_config,
    viz_config=viz_config,
    validation_config=validation_config,
    output_dir=str(output_dir)
)

# Let's see what our demonstration video looks like
display(HTML(f"""
<video width="640" height="480" controls>
    <source src="meme_tutorial.mp4" type="video/mp4">
    Your browser does not support the video tag.
</video>
"""))

## Training Our Meme Master
Now that we have our demonstration, let's train our agent to become a meme master:

In [ ]:
# Time to learn!
print("🎓 Training our agent to become a meme master...")
training_results = simulator.train_agents()

# Create a training visualization
plt.figure(figsize=(12, 6))

# Create subplots for different metrics
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot mean reward
ax1.bar(['Mean Reward'], [training_results['mean_reward']],
        color='#66B2FF', alpha=0.7)
ax1.set_ylim(0, 1.0)
ax1.set_title('Mean Reward')
ax1.grid(True, alpha=0.3)

# Plot success rate
ax2.bar(['Success Rate'], [training_results['success_rate']],
        color='#99FF99', alpha=0.7)
ax2.set_ylim(0, 1.0)
ax2.set_title('Success Rate')
ax2.grid(True, alpha=0.3)

plt.suptitle("Meme Master Training Results", fontsize=14)
plt.tight_layout()
plt.show()

# Print fun stats with emojis
print("\n🎯 Training Results:")
print(f"🎲 Average Reward: {training_results['mean_reward']:.2f}")
print(f"⭐ Success Rate: {training_results['success_rate']*100:.1f}%")
print(f"📊 Total Episodes: {training_results['total_episodes']}")
print(f"⏱️ Mean Episode Length: {training_results['mean_length']:.1f} steps")
print(f"📈 Reward Std Dev: {training_results['std_reward']:.2f}")

# Add some fun commentary based on the results
success_rate = training_results['success_rate']
if success_rate > 0.9:
    print("\n🏆 Wow! Your agent is a certified meme lord!")
elif success_rate > 0.7:
    print("\n🌟 Pretty good! Your agent is becoming a meme artisan!")
elif success_rate > 0.5:
    print("\n📚 Not bad! Your agent is learning the ways of the meme!")
else:
    print("\n🌱 Keep training! Even meme masters started somewhere!")

## Unleashing the Meme Generator
Let's use our trained agent to generate some memes!

In [ ]:
# Prepare some fun meme templates and captions
meme_tasks = [
    {
        "template": "distracted_boyfriend.jpg",
        "captions": [
            "Python",
            "Me",
            "JavaScript"
        ]
    },
    {
        "template": "drake.jpg",
        "captions": [
            "Writing code without comments",
            "Writing comments without code"
        ]
    },
    {
        "template": "expanding_brain.jpg",
        "captions": [
            "print('debug')",
            "console.log('debug')",
            "Using a debugger",
            "Adding random print statements and hoping for the best"
        ]
    }
]

# Generate memes!
print("🎨 Generating memes...")
for i, task in enumerate(meme_tasks, 1):
    print(f"\n✨ Creating meme {i}/{len(meme_tasks)}")
    print(f"Template: {task['template']}")
    print(f"Captions: {', '.join(task['captions'])}")

    try:
        # Reset the environment
        observation, info = simulator.reset()

        # Initialize metrics for this meme
        total_reward = 0
        steps = 0
        done = False
        truncated = False

        # Generate the meme step by step
        while not (done or truncated):
            # Generate action using the environment's action space
            action = simulator.action_space.sample()  # For demonstration

            # Take a step in the environment
            observation, reward, done, truncated, info = simulator.step(action)

            total_reward += reward
            steps += 1

            # Show progress
            if steps % 5 == 0:
                print(f"🔄 Step {steps}, Reward: {total_reward:.2f}")

            # Optional: Add task-specific actions based on the current step
            if 'ui_elements' in observation:
                for element in observation['ui_elements']:
                    if element['type'] == 'textbox' and task['captions']:
                        # Add caption to textbox
                        action = {
                            'action_type': 3,  # type action
                            'target_element': element['bbox'],
                            'parameters': {
                                'text_input': task['captions'][0],
                                'drag_end': [0, 0]
                            }
                        }
                        task['captions'] = task['captions'][1:]

            # Check for success or timeout
            if info.get('meme_generated') or steps >= 100:
                break

        # Show final results for this meme
        print(f"\n📊 Meme Generation Stats:")
        print(f"Total Steps: {steps}")
        print(f"Total Reward: {total_reward:.2f}")

        # Display the generated meme if available
        meme_path = output_dir / f"meme_{i}.png"
        if meme_path.exists():
            display(Image(filename=str(meme_path)))
        else:
            print("⚠️ Meme image not found - but that's okay! This is a demo.")

    except Exception as e:
        print(f"❌ Error generating meme {i}: {str(e)}")
        print("Don't worry! This is expected in the demo without a real UI.")
        continue

    print("\n" + "="*50 + "\n")

print("""
✨ Meme Complete! ✨""")

## Analyzing Our Meme Factory
Let's look at some fun statistics about our meme generation:

In [ ]:
# Get evaluation results
eval_results = simulator.evaluate_agents()

# Create a fun visualization of our meme factory stats
stats = {
    "Memes Generated": len(meme_tasks),
    "Success Rate": f"{eval_results['success_rate']*100:.1f}%",
    "Average Generation Time": f"{eval_results['mean_length']:.1f}s",
    "Quality Score": f"{eval_results['mean_reward']:.2f}/1.0"
}

print("📊 Meme Factory Statistics:")
for stat, value in stats.items():
    print(f"{stat}: {value}")

# Plot a fun pie chart of time spent on each step
steps = [
    "Finding Templates",
    "Adding Captions",
    "Adjusting Layout",
    "Generating Meme",
    "Saving Masterpiece"
]
times = [15, 30, 25, 20, 10]  # Example percentages

plt.figure(figsize=(10, 8))
plt.pie(times, labels=steps, autopct='%1.1f%%',
        colors=['#FF9999', '#66B2FF', '#99FF99', '#FFCC99', '#FF99CC'])
plt.title("Time Spent Making Memes")
plt.show()

# Conclusion
Congratulations! You've successfully created an automated meme factory! 🎉
Some fun things we learned:

- Our agent can learn to navigate UI elements and create memes
- The power of combining computer vision with reinforcement learning
- How to make our code more entertaining with emojis 😄

## Next Steps
Want to make your meme factory even better? Here are some fun ideas:

- Train on different meme templates
- Add text effects and styling
- Create a meme recommendation system
- Build a Discord bot using this automation
- Generate captions using Claude